#### Q1. Install MLflow

What's the version that you have?

In [1]:
import mlflow

print(mlflow.__version__)

2.13.0


#### Q2. Download and preprocess the data

How many files were saved to ```OUTPUT_FOLDER```?

In [2]:
!python preprocess_data.py --raw_data_path ./data --dest_path ./output

In [4]:
import os
print("files in output folder: ", len(os.listdir("./output")))

files in output folder:  4


#### Q3. Train a model with autolog

What is the value of the ```min_samples_split``` parameter?

In [6]:
!python train.py

2024/05/27 00:54:33 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/05/27 00:54:34 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/27 00:54:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [26]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)
run = client.get_run("3ba3100fa2cb4612861c0fb68e44bbf3")

min_samples_split = run.data.params.get("min_samples_split")

In [27]:
print("min_samples_split: ", min_samples_split)

min_samples_split:  2


#### Q4. Launch the tracking server locally

In addition to backend-store-uri, what else do you need to pass to properly configure the server?

* default-artifact-root

#### Q5. Tune model hyperparameters

What's the best validation RMSE that you got?

In [30]:
!python hpo.py

2024/05/27 01:21:41 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt-' does not exist. Creating a new experiment.
  0%|                                    | 0/15 [00:00<?, ?trial/s, best loss=?]/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

  7%|▋          | 1/15 [00:07<01:51,  7.97s/trial, best loss: 5.370086069268862]/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

 13%|█▍         | 2/15 [00:08<00:46,  3.55s/trial, best loss: 5.370086069268862]/home/codespace/anaconda3/e

In [31]:
# List all experiments
experiments = client.search_experiments()

# Print experiment details
for exp in experiments:
    print(f"Name: {exp.name}, ID: {exp.experiment_id}")

Name: random-forest-hyperopt-, ID: 3
Name: 02-homework, ID: 1
Name: Default, ID: 0


In [32]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='3',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.rmse ASC"]
)

In [33]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: f12239df118a40c297eab4b57616c2e8, rmse: 5.3354


#### Q6. Promote the best model to the model registry

What is the test RMSE of the best model?

In [37]:
!python register_model.py

2024/05/27 01:39:10 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/05/27 01:39:11 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/27 01:39:16 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [42]:
import os
import pickle
import mlflow
from sklearn.metrics import mean_squared_error

logged_model = 'runs:/b0c040c230a94ec78ce5c152b27035fc/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

sklearn_model = mlflow.sklearn.load_model(logged_model)

In [43]:
# Load the test dataset
data_path = "./output"
with open(os.path.join(data_path, "test.pkl"), "rb") as f_in:
    X_test, y_test = pickle.load(f_in)

# Make predictions on the test dataset
y_pred = loaded_model.predict(X_test)

# Evaluate the model's performance
test_rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Test RMSE: {test_rmse}")

Test RMSE: 5.567408012462019


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
